In [1]:
# ================================================
# 🧩 Celda 1 — Importaciones
# ================================================
import os
import math
import numpy as np
import pandas as pd

from scipy.stats import skew, normaltest
from sklearn.ensemble import IsolationForest

import sys
sys.path.append("../scripts")
sys.path.append("../datasets")

# --- Módulos propios del proyecto ---
from cargar_dataset import cargar_dataset                      # Función para cargar datasets según configuración
from config_datasets import config_datasets  
print("✅ Importaciones listas.")


✅ Importaciones listas.


In [2]:
# ================================================
# 🧩 Celda 2 — Integración con tu pipeline
# ================================================
# Requiere:
# - config_datasets: dict con la configuración de cada dataset (paths, columnas, etc.)
# - cargar_dataset: función existente en tu proyecto que retorna (X, y, info_adicional)

# Si tenés módulos locales, descomentá y ajustá:
# from config_datasets import config_datasets
# from cargar_dataset import cargar_dataset

# Lista de datasets a evaluar (claves presentes en tu config_datasets)
lista_nombres_datasets = [
    "glass",
    "ecoli",
    "heart",
    "wdbc",
    "shuttle",
]

# Hiperparámetros del detector de outliers
parametros_isolation_forest = {
    "contamination": 0.05,
    "random_state": 42,
    "n_estimators": 100
}

# Umbral de significancia para normalidad por columna (p-valor)
alfa_normalidad = 0.05

# Para evitar que SHUTTLE u otros gigantes demoren, podés muestrear:
usar_muestreo = True
tamano_muestra_maxima = 20000  # ajustá según tu máquina

print("✅ Integración: listo para usar cargar_dataset(config_datasets).")


✅ Integración: listo para usar cargar_dataset(config_datasets).


In [3]:
# ================================================
# 🧩 Celda 3 — Funciones (nombres explícitos en español)
# ================================================
def muestrear_si_corresponde(X, y, usar_muestreo, tamano_muestra_maxima, random_state=42):
    if not usar_muestreo:
        return X, y
    cantidad = int(len(y))
    if cantidad <= int(tamano_muestra_maxima):
        return X, y
    rng = np.random.RandomState(random_state)
    indices = np.arange(cantidad)
    rng.shuffle(indices)
    indices = indices[:int(tamano_muestra_maxima)]
    X_m = X.iloc[indices].reset_index(drop=True)
    y_m = y.iloc[indices].reset_index(drop=True)
    return X_m, y_m

def calcular_asimetria_promedio_absoluta(X):
    columnas_numericas = []
    for nombre_columna in X.columns:
        if np.issubdtype(X[nombre_columna].dtype, np.number):
            columnas_numericas.append(nombre_columna)

    valores_asimetria = []
    for nombre_columna in columnas_numericas:
        serie = X[nombre_columna].astype(float)
        valor_skew = skew(serie, bias=False, nan_policy='omit')
        if isinstance(valor_skew, float) and not math.isnan(valor_skew):
            valores_asimetria.append(abs(valor_skew))

    if len(valores_asimetria) == 0:
        return float("nan")

    suma = 0.0
    for v in valores_asimetria:
        suma = suma + v
    promedio = suma / float(len(valores_asimetria))
    return promedio

def calcular_porcentaje_columnas_normales(X, alfa):
    columnas_numericas = []
    for nombre_columna in X.columns:
        if np.issubdtype(X[nombre_columna].dtype, np.number):
            columnas_numericas.append(nombre_columna)

    total = 0
    normales = 0
    for nombre_columna in columnas_numericas:
        serie = X[nombre_columna].astype(float).dropna()
        if serie.shape[0] >= 8:
            try:
                estadistico, pvalor = normaltest(serie)
                total = total + 1
                if pvalor > alfa:
                    normales = normales + 1
            except Exception:
                # No sumo al total si falla
                pass

    if total == 0:
        return float("nan")
    porcentaje = (normales / float(total)) * 100.0
    return porcentaje

def calcular_correlacion_maxima_absoluta(X):
    Xn = X.select_dtypes(include=[np.number])
    if Xn.shape[1] < 2:
        return float("nan")
    matriz = Xn.corr(method="pearson").abs()
    # anular diagonal
    filas = matriz.shape[0]
    i = 0
    while i < filas:
        matriz.iat[i, i] = 0.0
        i = i + 1
    return float(matriz.values.max())

def calcular_porcentaje_outliers_if(X, params):
    Xn = X.select_dtypes(include=[np.number])
    if Xn.shape[1] == 0:
        return float("nan")
    modelo = IsolationForest(
        contamination=params.get("contamination", 0.05),
        random_state=params.get("random_state", 42),
        n_estimators=params.get("n_estimators", 100),
        n_jobs=1
    )
    etiquetas = modelo.fit_predict(Xn)
    total = int(len(etiquetas))
    outl = 0
    for e in etiquetas:
        if int(e) == -1:
            outl = outl + 1
    porcentaje = (outl / float(total)) * 100.0
    return porcentaje

# ================================================
# 🔧 Celda — Utilidades de robustez de tipos
# ================================================
import pandas as pd
import numpy as np

def asegurar_dataframe_features_y(X, y, cfg):
    """
    Si cargar_dataset devuelve arrays numpy, convierto a DataFrame/Series.
    Usa cfg['col_features'] y cfg['col_target'] cuando están disponibles.
    """
    # X -> DataFrame
    if isinstance(X, np.ndarray):
        # nombres de columnas
        if cfg.get("col_features") is not None:
            nombres_cols = list(cfg.get("col_features"))
        else:
            # fallback explícito
            ncols = int(X.shape[1])
            nombres_cols = []
            i = 0
            while i < ncols:
                nombres_cols.append(f"feature_{i}")
                i = i + 1
        X = pd.DataFrame(X, columns=nombres_cols)
    # y -> Series
    if isinstance(y, np.ndarray):
        # nombre de target
        nombre_target = cfg.get("col_target", "target")
        y = pd.Series(y, name=nombre_target)
    return X, y

def caracterizar_con_cargar_dataset(nombre_dataset, cfg, alfa, params, usar_muestreo, tamano_muestra_maxima):
    X, y, _ = cargar_dataset(
        path=cfg.get("path"),
        clase_minoria=cfg.get("clase_minoria"),
        col_features=cfg.get("col_features"),
        col_target=cfg.get("col_target"),
        sep=cfg.get("sep", ","),
        header=cfg.get("header", None),
        binarizar=cfg.get("binarizar", False),
        tipo=cfg.get("tipo", "tabular"),
        impute=cfg.get("impute", "median"),
        names=cfg.get("esquema") if cfg.get("header", None) is None else None,
    )

    # 👉 NUEVO: asegurar DataFrame/Series aunque cargar_dataset devuelva ndarrays
    X, y = asegurar_dataframe_features_y(X, y, cfg)

    # (resto de tu lógica igual)
    X, y = muestrear_si_corresponde(X, y, usar_muestreo, tamano_muestra_maxima)

    n_muestras = int(X.shape[0])
    n_features = int(X.shape[1])
    n_clases = int(len(pd.unique(y)))

    asim = calcular_asimetria_promedio_absoluta(X)
    porc_norm = calcular_porcentaje_columnas_normales(X, alfa)
    corr_max = calcular_correlacion_maxima_absoluta(X)
    porc_outl = calcular_porcentaje_outliers_if(X, params)

    if (not math.isnan(porc_outl) and porc_outl > 10.0) or (not math.isnan(porc_norm) and porc_norm < 30.0):
        sugerencia = "⚠️ Selectiva"
    elif not math.isnan(porc_outl) and porc_outl > 3.0:
        sugerencia = "🧹 Moderada"
    else:
        sugerencia = "❌ No"

    return {
        "Dataset": nombre_dataset,
        "N (muestras)": n_muestras,
        "# features": n_features,
        "# clases": n_clases,
        "% normalidad (K²)": None if math.isnan(porc_norm) else round(porc_norm, 1),
        "Asimetría promedio (|skew|)": None if math.isnan(asim) else round(asim, 2),
        "Correlación máx. (|r|)": None if math.isnan(corr_max) else round(corr_max, 2),
        "% outliers (IF)": None if math.isnan(porc_outl) else round(porc_outl, 1),
        "Limpieza recomendada": sugerencia
    }

def generar_tabla_resumen_con_pipeline(config_datasets, lista_nombres, alfa, params, usar_muestreo, tamano_muestra_maxima):
    resultados = []
    for nombre in lista_nombres:
        try:
            cfg = config_datasets.get(nombre)
            if cfg is None:
                raise KeyError(f"No existe config para '{nombre}' en config_datasets.")
            fila = caracterizar_con_cargar_dataset(
                nombre_dataset=nombre,
                cfg=cfg,
                alfa=alfa,
                params=params,
                usar_muestreo=usar_muestreo,
                tamano_muestra_maxima=tamano_muestra_maxima
            )
            resultados.append(fila)
        except Exception as e:
            resultados.append({
                "Dataset": nombre,
                "N (muestras)": None,
                "# features": None,
                "# clases": None,
                "% normalidad (K²)": None,
                "Asimetría promedio (|skew|)": None,
                "Correlación máx. (|r|)": None,
                "% outliers (IF)": None,
                "Limpieza recomendada": f"ERROR: {str(e)}"
            })
    tabla = pd.DataFrame(resultados)
    columnas = [
        "Dataset",
        "N (muestras)",
        "# features",
        "# clases",
        "% normalidad (K²)",
        "Asimetría promedio (|skew|)",
        "Correlación máx. (|r|)",
        "% outliers (IF)",
        "Limpieza recomendada"
    ]
    tabla = tabla[columnas]
    return tabla


In [4]:
# ================================================
# 📏 Celda — Estudio Kolmogorov–Smirnov (tabla aparte)
# ================================================
from scipy.stats import kstest

def calcular_estadistica_ks_por_columnas(X):
    """
    Para cada columna numérica:
      1) z-score (resto media, divido por std; si std==0, omito esa columna)
      2) kstest contra N(0,1)
    Devuelve lista de dicts por columna con p-valor y estadístico.
    """
    resultados = []
    Xn = X.select_dtypes(include=[np.number])
    for nombre_columna in Xn.columns:
        serie = Xn[nombre_columna].astype(float).dropna()
        if serie.shape[0] < 8:
            continue
        media = float(serie.mean())
        desvio = float(serie.std(ddof=1))
        if desvio == 0.0 or np.isnan(desvio):
            continue
        z = (serie - media) / desvio
        # KS contra normal estándar
        estadistico, pvalor = kstest(z, "norm")
        resultados.append({
            "columna": nombre_columna,
            "n": int(serie.shape[0]),
            "media": media,
            "std": desvio,
            "ks_stat": float(estadistico),
            "pvalor": float(pvalor)
        })
    return resultados

def resumen_ks_dataset(nombre_dataset, resultados_columnas, alfa=0.05):
    """
    Resume para un dataset: #cols evaluadas, #cols normales (p>alfa), %cols normales, p-min/mediana/max.
    """
    total = int(len(resultados_columnas))
    normales = 0
    pvalores = []
    i = 0
    while i < total:
        p = resultados_columnas[i]["pvalor"]
        pvalores.append(p)
        if p > alfa:
            normales = normales + 1
        i = i + 1

    if total == 0:
        return {
            "Dataset": nombre_dataset,
            "# cols evaluadas (KS)": 0,
            "# cols normales (KS)": 0,
            "% cols normales (KS)": "",
            "p-valor min": "",
            "p-valor mediana": "",
            "p-valor max": ""
        }

    p_min = float(np.min(pvalores))
    p_med = float(np.median(pvalores))
    p_max = float(np.max(pvalores))

    return {
        "Dataset": nombre_dataset,
        "# cols evaluadas (KS)": total,
        "# cols normales (KS)": normales,
        "% cols normales (KS)": round((normales / float(total)) * 100.0, 1),
        "p-valor min": round(p_min, 4),
        "p-valor mediana": round(p_med, 4),
        "p-valor max": round(p_max, 4)
    }

def generar_tabla_ks(config_datasets, lista_nombres, alfa=0.05, usar_muestreo=False, tamano_muestra_maxima=50000):
    """
    Genera:
      - tabla resumen por dataset
      - CSVs detallados por dataset (una fila por columna con ks_stat y p-valor)
    """
    filas_resumen = []
    carpeta_salida = "../resultados/caracterizacion/ks"
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida, exist_ok=True)

    for nombre in lista_nombres:
        try:
            cfg = config_datasets.get(nombre)
            if cfg is None:
                raise KeyError(f"No existe config para '{nombre}' en config_datasets.")

            X, y, _ = cargar_dataset(
                path=cfg.get("path"),
                clase_minoria=cfg.get("clase_minoria"),
                col_features=cfg.get("col_features"),
                col_target=cfg.get("col_target"),
                sep=cfg.get("sep", ","),
                header=cfg.get("header", None),
                binarizar=cfg.get("binarizar", False),
                tipo=cfg.get("tipo", "tabular"),
                impute=cfg.get("impute", "median"),
                names=cfg.get("esquema") if cfg.get("header", None) is None else None,
            )
            # Asegurar DF/Series
            X, y = asegurar_dataframe_features_y(X, y, cfg)

            # Muestreo opcional
            X, y = muestrear_si_corresponde(X, y, usar_muestreo, tamano_muestra_maxima)

            # KS por columnas
            resultados_cols = calcular_estadistica_ks_por_columnas(X)

            # Guardar detalle por columnas
            df_detalle = pd.DataFrame(resultados_cols)
            ruta_detalle = os.path.join(carpeta_salida, f"ks_detalle_{nombre}.csv")
            df_detalle.to_csv(ruta_detalle, index=False)

            # Resumen por dataset
            fila_resumen = resumen_ks_dataset(nombre, resultados_cols, alfa=alfa)
            filas_resumen.append(fila_resumen)

        except Exception as e:
            filas_resumen.append({
                "Dataset": nombre,
                "# cols evaluadas (KS)": "",
                "# cols normales (KS)": "",
                "% cols normales (KS)": "",
                "p-valor min": "",
                "p-valor mediana": "",
                "p-valor max": "",
                "ERROR": str(e)
            })

    tabla_resumen_ks = pd.DataFrame(filas_resumen)
    ruta_resumen = os.path.join(carpeta_salida, "ks_resumen.csv")
    tabla_resumen_ks.to_csv(ruta_resumen, index=False)
    return tabla_resumen_ks


In [5]:
# ================================================
# 🧩 Celda 4 — Ejecución y guardado
# ================================================
tabla_resumen = generar_tabla_resumen_con_pipeline(
    config_datasets=config_datasets,
    lista_nombres=lista_nombres_datasets,
    alfa=alfa_normalidad,
    params=parametros_isolation_forest,
    usar_muestreo=usar_muestreo,
    tamano_muestra_maxima=tamano_muestra_maxima
)

# Mostrar
print(tabla_resumen.to_markdown(index=False))

# Guardar
os.makedirs("../resultados/caracterizacion", exist_ok=True)
ruta_csv = "../resultados/caracterizacion/resumen_caracterizacion.csv"
ruta_md  = "../resultados/caracterizacion/resumen_caracterizacion.md"

tabla_resumen.to_csv(ruta_csv, index=False)

with open(ruta_md, "w", encoding="utf-8") as f:
    columnas = list(tabla_resumen.columns)
    f.write("| " + " | ".join(columnas) + " |\n")
    alineacion = []
    i = 0
    while i < len(columnas):
        if i == 0:
            alineacion.append(":---")
        else:
            alineacion.append(":---:")
        i = i + 1
    f.write("|" + "|".join(alineacion) + "|\n")
    for _, fila in tabla_resumen.iterrows():
        valores = []
        for c in columnas:
            v = fila[c]
            if v is None:
                valores.append("")
            else:
                valores.append(str(v))
        f.write("| " + " | ".join(valores) + " |\n")

print(f"\n✅ CSV: {ruta_csv}")
print(f"✅ MD : {ruta_md}")


| Dataset   |   N (muestras) |   # features |   # clases |   % normalidad (K²) |   Asimetría promedio (|skew|) |   Correlación máx. (|r|) |   % outliers (IF) | Limpieza recomendada   |
|:----------|---------------:|-------------:|-----------:|--------------------:|------------------------------:|-------------------------:|------------------:|:-----------------------|
| glass     |            214 |            9 |          6 |                   0 |                          2.07 |                     0.81 |               5.1 | ⚠️ Selectiva           |
| ecoli     |            336 |            7 |          8 |                   0 |                          3.65 |                     0.81 |               5.1 | ⚠️ Selectiva           |
| heart     |            303 |           13 |          5 |                   0 |                          0.78 |                     0.58 |               5.3 | ⚠️ Selectiva           |
| wdbc      |            569 |           30 |          2 |                 

In [6]:
# ================================================
# ▶️ Celda — Ejecutar KS y guardar tablas
# ================================================
# Usá la misma lista_nombres_datasets que ya tenés
tabla_ks = generar_tabla_ks(
    config_datasets=config_datasets,
    lista_nombres=lista_nombres_datasets,
    alfa=0.05,
    usar_muestreo=True,             # para acelerar Shuttle si es grande
    tamano_muestra_maxima=20000
)

print(tabla_ks.to_markdown(index=False))
print("\n✅ Guardado resumen KS en: ../resultados/caracterizacion/ks/ks_resumen.csv")
print("✅ Guardado detalle por columnas en: ../resultados/caracterizacion/ks/ks_detalle_*.csv")


| Dataset   |   # cols evaluadas (KS) |   # cols normales (KS) |   % cols normales (KS) |   p-valor min |   p-valor mediana |   p-valor max |
|:----------|------------------------:|-----------------------:|-----------------------:|--------------:|------------------:|--------------:|
| glass     |                       9 |                      0 |                    0   |             0 |            0      |        0.042  |
| ecoli     |                       7 |                      1 |                   14.3 |             0 |            0.0003 |        0.2114 |
| heart     |                      13 |                      3 |                   23.1 |             0 |            0      |        0.2992 |
| wdbc      |                      30 |                      5 |                   16.7 |             0 |            0      |        0.4608 |
| shuttle   |                       9 |                      0 |                    0   |             0 |            0      |        0      |

✅ Gua